In [ ]:
#changed/added screens

In [1]:
##########  importing libraries  ##########
import tkinter as tk
from tkinter import *
import cv2
import csv
import os
import numpy as np
from PIL import Image,ImageTk
import pandas as pd
import datetime
import time
from tkinter import messagebox

#s=StringVar()
#status = Label(window, textvariable=s,font=('times',12),fg='#444444')
#status.grid(row=8,columnspan=8,sticky='W',pady=(10,0))
#s.set('status')

##########  Enter Button  ##########

##########  manually fill log  ##########
def manually_fill():
    global sb
    sb = tk.Tk()
    sb.iconbitmap('AMS.ico')
    sb.title("Manually Fill Log")       

    def fill_attendance():
        ts = time.time()
        Date = datetime.datetime.fromtimestamp(ts).strftime('%Y_%m_%d')
        timeStamp = datetime.datetime.fromtimestamp(ts).strftime('%H:%M:%S')
        Time = datetime.datetime.fromtimestamp(ts).strftime('%H:%M:%S')
        Hour, Minute, Second = timeStamp.split(":")
        #Creatting csv of attendance

        #Create table for Attendance
        date_for_DB = datetime.datetime.fromtimestamp(ts).strftime('%Y_%m_%d')
        global subb
        subb=SUB_ENTRY.get()
        DB_table_name = str(subb + "_" + Date + "_Time_" + Hour + "_" + Minute + "_" + Second)

        import pymysql.connections              ###Connect to the database
        try:
            global cursor
            connection = pymysql.connect(host='localhost', user='root', password='', db='manually_fill_attendance')
            cursor = connection.cursor()
        except Exception as e:
            print(e)

        sql = "CREATE TABLE " + DB_table_name + """
                        (ID INT NOT NULL AUTO_INCREMENT,
                         ENROLLMENT varchar(100) NOT NULL,
                         NAME VARCHAR(50) NOT NULL,
                         DATE VARCHAR(20) NOT NULL,
                         TIME VARCHAR(20) NOT NULL,
                             PRIMARY KEY (ID)
                             );
                        """

        try:
            cursor.execute(sql)               ##for create a table
        except Exception as ex:
            print(ex)

            #### message box
        if subb == '':
            messagebox.showerror( message='Please enter a valid room number!',icon='error')
        else:
            sb.destroy()
            
            MFW = tk.Tk()
            MFW.iconbitmap('AMS.ico')
            MFW.title("Manual log of "+ str(subb))

                
            def testVal(inStr, acttyp):
                if acttyp == '1':  # insert
                    if not inStr.isdigit():
                        return False
                return True

            ENR = tk.Label(MFW, text="Employee ID :",  font=('times', 15))
            ENR.grid(row=0,column=0,sticky=('S','W'),padx=10,pady=(10,0))

            STU_NAME = tk.Label(MFW, text="Employee name :",font=('times', 15))
            STU_NAME.grid(row=2,column=0,sticky=('S','W'),padx=10,pady=(10,0))

            global ENR_ENTRY
            ENR_ENTRY = tk.Entry(MFW,width=20,validate='key', bg="white", font=('times', 23))
            ENR_ENTRY['validatecommand'] = (ENR_ENTRY.register(testVal), '%P', '%d')
            ENR_ENTRY.grid(row=1,column=0,padx=(10,0),pady=(3,10))

            def remove_enr():
                ENR_ENTRY.delete(first=0, last=22)

            STUDENT_ENTRY = tk.Entry(MFW, width=20, bg="white", font=('times', 23))
            STUDENT_ENTRY.grid(row=3,column=0,padx=(10,0),pady=(3,10))

            def remove_student():
                STUDENT_ENTRY.delete(first=0, last=22)

            ####get important variable
            def enter_data_DB():
                ENROLLMENT = ENR_ENTRY.get()
                STUDENT = STUDENT_ENTRY.get()
                if ENROLLMENT=='':
                    messagebox.showerror( message='Please enter ID!',icon='error')
                elif STUDENT=='':
                    messagebox.showerror( message='Please enter Name!',icon='error')
                else:
                    time = datetime.datetime.fromtimestamp(ts).strftime('%H:%M:%S')
                    Hour, Minute, Second = time.split(":")
                    Insert_data = "INSERT INTO " + DB_table_name + " (ID,ENROLLMENT,NAME,DATE,TIME) VALUES (0, %s, %s, %s,%s)"
                    VALUES = (str(ENROLLMENT), str(STUDENT), str(Date), str(time))
                    try:
                        cursor.execute(Insert_data, VALUES)
                    except Exception as e:
                        print(e)
                    ENR_ENTRY.delete(first=0, last=22)
                    STUDENT_ENTRY.delete(first=0, last=22)

            def create_csv():
                import csv
            
                cursor.execute("select * from " + DB_table_name + ";")
                csv_name='E:/Attendace managemnt system/Attendance/Manually Attendance/'+DB_table_name+'.csv'
                with open(csv_name, "w") as csv_file:
                    csv_writer = csv.writer(csv_file)
                    csv_writer.writerow([i[0] for i in cursor.description])  # write headers
                    csv_writer.writerows(cursor)
                    O="CSV created Successfully"
                    Notifi.configure(text=O, bg="Green", fg="white", width=33, font=('times', 19))
                    Notifi.place(x=180, y=380)
                    
                import csv
                import tkinter
                root = tkinter.Tk()
                root.title("log of " + subb)
                with open(csv_name, newline="") as file:
                    reader = csv.reader(file)
                    r = 0
                    for col in reader:
                        c = 0
                        for row in col:
                            # i've added some styling
                            label = tkinter.Label(root, width=13, height=1, fg="black", font=('times', 13, ' bold '),
                                                  bg="white", text=row, relief=tkinter.RIDGE)
                            label.grid(row=r, column=c)
                            c += 1
                        r += 1
                root.mainloop()

            Notifi = tk.Label(MFW, text="CSV created Successfully", font=('times', 19))


            c1ear_enroll = tk.Button(MFW, text="clear", command=remove_enr,activebackground='#999999',bg='#8894C3',borderwidth=0,font=('times', 15, ' bold '))
            c1ear_enroll.grid(row=1,column=1,padx=(0,10),sticky='W',pady=(3,10))

            c1ear_student = tk.Button(MFW, text="clear", command=remove_student,activebackground='#999999',bg='#8894C3',borderwidth=0, font=('times', 15, ' bold '))
            c1ear_student.grid(row=3,column=1,padx=(0,10),sticky='W',pady=(3,10))
        ###

            DATA_SUB = tk.Button(MFW, text="Add more",command=enter_data_DB,width=13,activebackground='#999999',bg='#8894C3',borderwidth=1, font=('times', 15))
            DATA_SUB.grid(row=4,column=0,padx=10,pady=10)

            MAKE_CSV = tk.Button(MFW, text="Convert to CSV",command=create_csv,width=13,activebackground='#999999',bg='#8894C3',borderwidth=1,font=('times', 15))
            MAKE_CSV.grid(row=4,column=1,padx=10,pady=10)
            
            MFW.mainloop()

    SUB = tk.Label(sb, text="Enter room number", font=('times', 15))
    SUB.grid(row=0,column=0,sticky=('S','W'),padx=10,pady=(10,0))

    global SUB_ENTRY

    SUB_ENTRY = tk.Entry(sb, width=20, bg="white", fg="black", font=('times', 23))
    SUB_ENTRY.grid(row=1,column=0,padx=10,pady=(0,10))

    fill_manual_attendance = tk.Button(sb, text="Fill log",command=fill_attendance, width=13,activebackground='#999999',bg='#8894C3',borderwidth=1, font=('times', 15))
    fill_manual_attendance.grid(row=2,column=0,sticky=('E'),padx=10,pady=5)
    
    sb.mainloop()

##For clear textbox
def clearall():
    txt.delete(first=0, last=22)
    txt2.delete(first=0, last=22)
    s.set("click on 'Take Images' to proceed")
    
    
def clear():
    txt.delete(first=0, last=22)
    
def clear1():
    txt2.delete(first=0, last=22)

def take_img():
    l1 = txt.get()
    l2 = txt2.get()
    txt.delete(first=0, last=22)
    txt2.delete(first=0, last=22)
    
    if l1 == '':
        messagebox.showerror( message='Please enter ID!',icon='error')         #### add message boxes
    elif l2 == '':
        messagebox.showerror( message='Please enter Name!',icon='error')
    else:
        try:
            messagebox.showinfo(message = "Imma take your pic now.. aight?")
            cam = cv2.VideoCapture(0)
            detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
            Enrollment = txt.get()
            Name = txt2.get()
            sampleNum = 0
            while (True):
                ret, img = cam.read()
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                faces = detector.detectMultiScale(gray, 1.3, 5)
                for (x, y, w, h) in faces:
                    cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
                    # incrementing sample number
                    sampleNum = sampleNum + 1
                    # saving the captured face in the dataset folder
                    cv2.imwrite("TrainingImage/" + Name + "." + Enrollment + '.' + str(sampleNum) + ".jpg",
                                gray[y:y + h, x:x + w])
                    cv2.imshow('Frame', img)
                # wait for 100 miliseconds
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                # break if the sample number is morethan 100
                elif sampleNum > 70:
                    break
            cam.release()
            cv2.destroyAllWindows()
            ts = time.time()
            Date = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d')
            Time = datetime.datetime.fromtimestamp(ts).strftime('%H:%M:%S')
            row = [Enrollment, Name, Date,Time]
            with open('StudentDetails\StudentDetails.csv', 'a+') as csvFile:
                writer = csv.writer(csvFile, delimiter=',')
                writer.writerow(row)
                csvFile.close()     
            s.set("click on 'Take Images' to proceed")
            
        except FileExistsError as F:
            messagebox.showerror( message='Employee data already exists!',icon='error')

#automatic fill log

def subjectchoose():
    def Fillattendances():
        sub=tx.get()
        now = time.time()  ###For calculate seconds of video
        future = now + 10           ############## changed 20 to 10
        if time.time() < future:
            if sub == '':
                messagebox.showerror( message='Please enter room number!',icon='error')     ##add error message instead
            else:
                recognizer = cv2.face.LBPHFaceRecognizer_create()  # cv2.createLBPHFaceRecognizer()
                try:
                    recognizer.read("TrainingImageLabel\Trainner.yml")
                except:
                    messagebox.showerror( message='Model not found, please train model first!',icon='error')
                    
                harcascadePath = "haarcascade_frontalface_default.xml"
                faceCascade = cv2.CascadeClassifier(harcascadePath)
                df = pd.read_csv("StudentDetails\StudentDetails.csv")
                cam = cv2.VideoCapture(0)
                font = cv2.FONT_HERSHEY_SIMPLEX
                col_names = ['Enrollment', 'Name', 'Date', 'Time']
                attendance = pd.DataFrame(columns=col_names)
                while True:
                    ret, im = cam.read()
                    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
                    faces = faceCascade.detectMultiScale(gray, 1.2, 5)
                    for (x, y, w, h) in faces:
                        global Id

                        Id, conf = recognizer.predict(gray[y:y + h, x:x + w])
                        if (conf <70):
                            print(conf)
                            global Subject
                            global aa
                            global date
                            global timeStamp
                            Subject = tx.get()
                            ts = time.time()
                            date = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d')
                            timeStamp = datetime.datetime.fromtimestamp(ts).strftime('%H:%M:%S')
                            aa = df.loc[df['Enrollment'] == Id]['Name'].values
                            global tt
                            tt = str(Id) + "-" + aa
                            En = '15624031' + str(Id)
                            attendance.loc[len(attendance)] = [Id, aa, date, timeStamp]
                            cv2.rectangle(im, (x, y), (x + w, y + h), (0, 260, 0), 7)
                            cv2.putText(im, str(tt), (x + h, y), font, 1, (255, 255, 0,), 4)

                        else:
                            Id = 'Unknown'
                            tt = str(Id)
                            cv2.rectangle(im, (x, y), (x + w, y + h), (0, 25, 255), 7)
                            cv2.putText(im, str(tt), (x + h, y), font, 1, (0, 25, 255), 4)
                    if time.time() > future:
                        break

                    attendance = attendance.drop_duplicates(['Enrollment'], keep='first')
                    cv2.imshow('Filling attedance..', im)
                    key = cv2.waitKey(20) & 0xff       ########################30 to 20
                    if key == 27:
                        break

                ts = time.time()
                date = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d')
                timeStamp = datetime.datetime.fromtimestamp(ts).strftime('%H:%M:%S')
                Hour, Minute, Second = timeStamp.split(":")
                fileName = "Attendance/" + Subject + "_" + date + "_" + Hour + "-" + Minute + "-" + Second + ".csv"
                attendance = attendance.drop_duplicates(['Enrollment'], keep='first')
                #print(attendance)
                attendance.to_csv(fileName, index=False)

                ##Create table for Attendance
                date_for_DB = datetime.datetime.fromtimestamp(ts).strftime('%Y_%m_%d')
                DB_Table_name = str( Subject + "_" + date_for_DB + "_Time_" + Hour + "_" + Minute + "_" + Second)
                
                import pymysql.connections

                ###Connect to the database
                try:
                    global cursor
                    connection = pymysql.connect(host='localhost', user='root', password='', db='Face_reco_fill')
                    cursor = connection.cursor()
                except Exception as e:
                    print(e)

                sql = "CREATE TABLE " + DB_Table_name + "(ID INT NOT NULL AUTO_INCREMENT,ENROLLMENT varchar(100) NOT NULL,NAME VARCHAR(50) NOT NULL,DATE VARCHAR(20) NOT NULL,TIME VARCHAR(20) NOT NULL,PRIMARY KEY (ID));"
                ####Now enter attendance in Database
                insert_data =  "INSERT INTO " + DB_Table_name + " (ID,ENROLLMENT,NAME,DATE,TIME) VALUES (0, %s, %s, %s,%s)"
                VALUES = (str(Id), str(aa), str(date), str(timeStamp))
                try:
                    cursor.execute(sql)  ##for create a table
                    cursor.execute(insert_data, VALUES)##For insert data into table
                except Exception as ex:
                    print(ex)  #

                    ####### create status bar  for This#####
                M = 'log filled Successfully'
                #Notifica.configure(text=M, bg="Green", fg="white", width=33, font=('times', 15, 'bold'))
                #Notifica.place(x=20, y=250)

                cam.release()
                cv2.destroyAllWindows()

                import csv
                import tkinter
                root = tkinter.Tk()
                root.title("Log of " + Subject)
                cs = 'E:/Attendace managemnt system/' + fileName
                with open(cs, newline="") as file:
                    reader = csv.reader(file)
                    r = 0
                    for col in reader:
                        c = 0
                        for row in col:
                            # i've added some styling
                            label = tkinter.Label(root, width=8, height=1, fg="black", font=('times', 15),
                                                  bg="white", text=row, relief=tkinter.RIDGE)
                            label.grid(row=r, column=c)
                            c += 1
                        r += 1
                root.mainloop()
                print(attendance)

    ###windo is frame for subject chooser
    windo = tk.Tk()
    windo.iconbitmap('AMS.ico')
    windo.title("Automatically fill log")

    sub = tk.Label(windo, text="Room Number",  font=('times', 15))
    sub.grid(row=0,column=0,sticky=('W','S'),padx=10,pady=(10,0))

    tx = tk.Entry(windo, width=20, font=('times', 23))
    tx.grid(row=1,column=0,padx=10,pady=(3,5))

    fill_a = tk.Button(windo, text="Register",command=Fillattendances ,width=13,activebackground='#999999',bg='#8894C3',borderwidth=1, font=('times', 15))
    fill_a.grid(row=2,column=0,sticky='E',padx=10,pady=10)
    
    windo.mainloop()


def admin_panel():
    win = tk.Tk()
    win.iconbitmap('AMS.ico')
    win.title("Admin Login")

    def log_in():
        username = un_entr.get()
        password = pw_entr.get()

        if username == 'k' :
            if password == 'k':
                win.destroy()
                
                import csv
                import tkinter
                root = tkinter.Tk()
                root.title("Employee Details")

                cs = 'E:/Attendace managemnt system/StudentDetails/StudentDetails.csv'
                with open(cs, newline="") as file:
                    reader = csv.reader(file)
                    r = 0
                    for col in reader:
                        c = 0
                        for row in col:
                            label = tkinter.Label(root, width=8, font=('times', 15), text=row, relief=tkinter.RIDGE)
                            label.grid(row=r, column=c)
                            c += 1
                        r += 1
                root.mainloop()
            else:
                messagebox.showerror( message='Incorrecct ID or Password!',icon='error')

        else:
            messagebox.showerror( message='Please enter Name & ID!',icon='error')

    un = tk.Label(win, text="Enter username :",font=('times', 15))
    un.grid(row=0,column=0,sticky=('S','W'),padx=10,pady=(10,0))

    pw = tk.Label(win, text="Enter password :", font=('times', 15))
    pw.grid(row=2,column=0,sticky=('S','W'),padx=10,pady=(10,0))

    def c00():
        un_entr.delete(first=0, last=22)

    un_entr = tk.Entry(win, width=20, font=('times', 23))
    un_entr.grid(row=1,column=0,padx=(10,0),pady=(3,10))

    def c11():
        pw_entr.delete(first=0, last=22)

    pw_entr = tk.Entry(win, width=20,show="*",  font=('times', 23))
    pw_entr.grid(row=3,column=0,padx=(10,0),pady=(3,10))
    
    c0 = tk.Button(win, text="Clear", command=c00,width=13,activebackground='#999999',bg='#8894C3',borderwidth=0, font=('times', 15))
    c0.grid(row=1,column=1,sticky=('W'),pady=(3,10),padx=(0,10))

    c1 = tk.Button(win, text="Clear", command=c11,width=13 ,activebackground='#999999',bg='#8894C3',borderwidth=0, font=('times', 15))
    c1.grid(row=3,column=1,sticky=('W'),pady=(3,10),padx=(0,10))

    Login = tk.Button(win, text="Login", command=log_in,width=13,activebackground='#999999',bg='#8894C3',borderwidth=1, font=('times', 15))
    Login.grid(row=4,column=0,sticky='E',padx=10,pady=10)
    
    win.mainloop()
            
#######    For train the model     ########

def trainimg():
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    global detector
    detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    try:
        global faces,Id
        faces, Id = getImagesAndLabels("TrainingImage")
    except Exception as e:
        messagebox.showinfo( message='Please make "TrainingImage" folder!',icon='info')
        
    recognizer.train(faces, np.array(Id))
    try:
        recognizer.save("TrainingImageLabel\Trainner.yml")
    except Exception as e:
        messagebox.showinfo( message='Please make "TrainingImageLabel" folder!',icon='info')

def getImagesAndLabels(path):
    imagePaths = [os.path.join(path, f) for f in os.listdir(path)]
    # create empth face list
    faceSamples = []
    # create empty ID list
    Ids = []
    # now looping through all the image paths and loading the Ids and the images
    for imagePath in imagePaths:
        # loading the image and converting it to gray scale
        pilImage = Image.open(imagePath).convert('L')
        # Now we are converting the PIL image into numpy array
        imageNp = np.array(pilImage, 'uint8')
        # getting the Id from the image

        Id = int(os.path.split(imagePath)[-1].split(".")[1])
        # extract the face from the training image sample
        faces = detector.detectMultiScale(imageNp)
        # If a face is there then append that in the list as well as Id of it
        for (x, y, w, h) in faces:
            faceSamples.append(imageNp[y:y + h, x:x + w])
            Ids.append(Id)
    return faceSamples, Ids

#############################
def new_register():
    global txt,txt2
    reg = tk.Tk()
    reg.title("FSMS - Register")
    reg.configure(padx=10,pady=10)
    
    s = StringVar()
    status=tk.Label(reg,textvariable=s,font=('times',12),fg='#444444')
    status.grid(row=8,columnspan=8)
    s.set("")

    lbl2 = tk.Label(reg, text="Name", font=('times', 15))
    lbl2.grid(row=0,column=0,sticky='W')

    txt2 = tk.Entry(reg, width=20, bg="white",font=('times', 20))
    txt2.grid(row=1,column=0)
    
    clearButton = tk.Button(reg,text="clear",command=clear,activebackground='#999999',bg='#8894C3',borderwidth=0,font=('times',15))
    clearButton.grid(row=1,column=1,sticky='W',pady=5)
    
    lbl = tk.Label(reg, text="Identification No.", font=('times', 15))
    lbl.grid(row=2, column=0,sticky='W')

    def testVal(inStr, acttyp):
        if acttyp == '1': #insert
            if not inStr.isdigit():
                return False
        return True

    txt = tk.Entry(reg, validate="key", width=20, bg="white", font=('times', 20))
    txt['validatecommand'] = (txt.register(testVal),'%P','%d')
    txt.grid(row=3,column=0)

    clearButton1 = tk.Button(reg,text="clear",command=clear1,activebackground='#999999',bg='#8894C3',borderwidth=0,font=('times',15))
    clearButton1.grid(row=3,column=1,sticky='W',pady=5)
    
    lbl3 = tk.Label(reg, text="Rooms Accesible", font=('times', 15))
    lbl3.grid(row=4,column=0,sticky='W')
    lbl4 = tk.Label(reg, text="separated by commas. ex (123,101,222)", font=('times', 8))
    
    clearButton = tk.Button(reg,text="clear",command=clear,activebackground='#999999',bg='#8894C3',borderwidth=0,font=('times',15))
    clearButton.grid(row=5,column=1,sticky='W',pady=5)

    txt3 = tk.Entry(reg, width=20, bg="white",font=('times', 20))
    txt3.grid(row=5,column=0)
    
    #EnterBtn = tk.Button(reg, text="Register",width=13, command =clearall,activebackground='#999999',bg='#b9b9b9',borderwidth=1,font=('times',15))
    #EnterBtn.grid(row=6,column=0,sticky='E',pady=5)

    takeImg = tk.Button(reg, text="Register",width=13,command=take_img,activebackground='#999999',bg='#8894C3',borderwidth=1,font=('times', 15))
    takeImg.grid(row=7,column=0,pady=5,padx=10)

    trainImg = tk.Button(reg, text="Train Model",width=13,command=trainimg,activebackground='#999999',bg='#8894C3',borderwidth=1,font=( 'times',15))
    trainImg.grid(row=7,column=1,pady=5,padx=10)

    reg.resizable(False, False)
    reg.iconbitmap('AMS.ico')
    reg.mainloop()

def vew():
    vis = tk.Tk()
    vis.title("FSMS - Employee Record")
    vis.configure(padx=10,pady=10)
    
    s = StringVar()
    status=tk.Label(vis,textvariable=s,font=('times',12),fg='#444444')
    status.grid(row=8,columnspan=8)
    s.set("")
    
    vis.resizable(False, False)
    vis.iconbitmap('AMS.ico')
    vis.mainloop()
    
def visitor():
    vis = tk.Tk()
    vis.title("FSMS - Visitor Registry")
    vis.configure(padx=10,pady=10)
    
    s = StringVar()
    status=tk.Label(vis,textvariable=s,font=('times',12),fg='#444444')
    status.grid(row=8,columnspan=8)
    s.set("")

    lbl2 = tk.Label(vis, text="Name", font=('times', 15))
    lbl2.grid(row=0,column=0,sticky='W')

    txt2 = tk.Entry(vis, width=20, bg="white",font=('times', 20))
    txt2.grid(row=1,column=0)
    
    clearButton = tk.Button(vis,text="clear",command=clear,activebackground='#999999',bg='#8894C3',borderwidth=0,font=('times',15))
    clearButton.grid(row=1,column=2,sticky='W',pady=5)
    
    lbl = tk.Label(vis, text="Identification No.", font=('times', 15))
    lbl.grid(row=2, column=0,sticky='W')

    def testVal(inStr, acttyp):
        if acttyp == '1': #insert
            if not inStr.isdigit():
                return False
        return True

    txt = tk.Entry(vis, validate="key", width=20, bg="white", font=('times', 20))
    txt['validatecommand'] = (txt.register(testVal),'%P','%d')
    txt.grid(row=3,column=0)

    clearButton1 = tk.Button(vis,text="clear",command=clear1,activebackground='#999999',bg='#8894C3',borderwidth=0,font=('times',15))
    clearButton1.grid(row=3,column=1,sticky='W',pady=5)
    
    lbl3 = tk.Label(vis, text="Rooms Accesible", font=('times', 15))
    lbl3.grid(row=4,column=0,sticky='W')
    lbl4 = tk.Label(vis, text="separated by commas. ex (123,101,222)", font=('times', 8))
    
    clearButton = tk.Button(vis,text="clear",command=clear,activebackground='#999999',bg='#8894C3',borderwidth=0,font=('times',15))
    clearButton.grid(row=5,column=1,sticky='W',pady=5)

    txt3 = tk.Entry(vis, width=20, bg="white",font=('times', 20))
    txt3.grid(row=5,column=0)
    
    #EnterBtn = tk.Button(reg, text="Register",width=13, command =clearall,activebackground='#999999',bg='#b9b9b9',borderwidth=1,font=('times',15))
    #EnterBtn.grid(row=6,column=0,sticky='E',pady=5)

    takeImg = tk.Button(vis, text="Register",width=13,command=take_img,activebackground='#999999',bg='#8894C3',borderwidth=1,font=('times', 15))
    takeImg.grid(row=7,column=0,pady=5,padx=10)

    trainImg = tk.Button(vis, text="Train Model",width=13,command=trainimg,activebackground='#999999',bg='#8894C3',borderwidth=1,font=( 'times',15))
    trainImg.grid(row=7,column=1,pady=5,padx=10)

    vis.resizable(False, False)
    vis.iconbitmap('AMS.ico')
    vis.mainloop()
    
##########  Main frame  ##########

window = tk.Tk()
window.title("FSMS-Face Recognition Based Security Management System")
window.configure(padx=10,pady=10)

s = StringVar()
status=tk.Label(window,textvariable=s,font=('times',12),fg='#444444')
status.grid(row=8,columnspan=8)
s.set("")

frm = Frame(window,bd=4,bg="#aaaaaa")

#table


frm.grid(row=0,column=0,rowspan=5,padx=10,pady=5)

label = tk.Label(frm,text="please work lol")
label.grid(row=0,column=0)

FA = tk.Button(window, text="Login",width=13,command=subjectchoose,activebackground='#999999',bg='#8894C3',borderwidth=1,font=('times',15 ))
FA.grid(row=0,column=1,pady=5,padx=10)

new_registerBtn = tk.Button(window, text="New Register",width=13,command=new_register,activebackground='#999999',bg='#8894C3',borderwidth=1,font=('times',15 ))
new_registerBtn.grid(row=1,column=1,pady=5,padx=10)

Visitor = tk.Button(window, text="Visitor",width=13,command=visitor,activebackground='#999999',bg='#8894C3',borderwidth=1,font=('times',15 ))
Visitor.grid(row=2,column=1,pady=5,padx=10)

AP = tk.Button(window, text="Log Book",command=admin_panel,width=13,activebackground='#999999',bg='#8894C3',borderwidth=1,font=('times',15 ))
AP.grid(row=3,column=1,pady=10,padx=(45,10))

VEW = tk.Button(window, text="Employee Record",width=13,command=vew,activebackground='#999999',bg='#8894C3',borderwidth=1,font=('times',15 ))
VEW.grid(row=4,column=1,pady=5,padx=10)

#quitWindow = tk.Button(window, text="Manual Log", command=manually_fill, activebackground='#999999',width=13,bg='#b9b9b9',borderwidth=1,font=('times',15))
#quitWindow.grid(row=4,column=3,pady=5,padx=(45,10))

window.resizable(False, False)
window.iconbitmap('AMS.ico')
window.mainloop()